In [24]:
#import sys
#print(sys.version)

In [25]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [26]:
import ultralytics
ultralytics.__version__

'8.3.9'

In [27]:
# Not training the model, just using pretrained model trained on COCO dataset. 
# Aquest dataset té som vehicles a part d'altres objectes.
#Fer servir aquest model per la detecció i després track i count
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov5n.pt')

PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [28]:
#list of COCO dataset classes
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [29]:
#creant una instancia de tracker que ve de from tracker import*
tracker=Tracker()
count=0

In [30]:
#llegir el video
cap=cv2.VideoCapture('output2.mp4')

In [31]:
#VARIABLES PER GUARDAR COTXES UP I DOWN
down={}
up={}

counter_down=[]
counter_up=[]

In [32]:
import numpy as np
import cv2

# Definir las coordenadas del polígono de ROI
pts = np.array([[359, 340], [186, 371], [8, 953], [446, 951]], np.int32)

# Crear una máscara con las mismas dimensiones que el frame
mask = np.zeros((480, 270), dtype=np.uint8)  # Ajustar a la resolución de frame_redimensionado

# Dibujar el polígono en la máscara
cv2.fillPoly(mask, [pts], 255)  # El polígono será blanco (valor 255) en la máscara

import time

# Initialize the FPS counter
fps = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    count += 1
    original_height, original_width = frame.shape[:2]
    
    # Reducir la resolución del frame
    frame_resized = cv2.resize(frame, (270, 480))

    # Aplicar el ROI usando la máscara
    frame_roi = cv2.bitwise_and(frame_resized, frame_resized, mask=mask)

    # Saltar 1 de cada 2 frames
    if count % 2 != 0:
        fps += 1
        end_time = time.time()
        elapsed_time = end_time - start_time
        if elapsed_time > 0:
            current_fps = fps / elapsed_time
            cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # línea roja
            cv2.line(frame, (0, 450), (850, 450), (255, 0, 0), 3)  # línea azul
        cv2.imshow("frames", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
        continue

    # YOLO Prediction solo en la ROI
    results = model.predict(frame_roi)

    # Obtener los datos de la bounding box y la confianza
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # Solo trabajar con la clase "car"
    car_bboxes = []
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            # Ajustar las coordenadas para que se correspondan con el frame original
            x1 = int(x1 * (original_width / frame_resized.shape[1]))  # Ajuste para el ancho original
            x2 = int(x2 * (original_width / frame_resized.shape[1]))
            y1 = int(y1 * (original_height / frame_resized.shape[0]))  # Ajuste para la altura original
            y2 = int(y2 * (original_height / frame_resized.shape[0]))
            car_bboxes.append([x1, y1, x2, y2])

    # Proveer las coordenadas al tracker
    bbox_id = tracker.update(car_bboxes)

    # Dibujar las bounding boxes en el frame original
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) // 2)
        cy = int((y3 + y4) // 2)

        # Dibuja la bounding box y el ID
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {id}', (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Actualizar contadores hacia abajo/arriba
        red_line_y = 400
        blue_line_y = 450
        offset = 7

        # Condición hacia abajo: primero cruzar la línea roja, luego la azul
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
            down[id] = cy
        if id in down:
            if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punto rojo
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                counter_down.append(id)  # lista

        # Condición hacia arriba: primero cruzar la línea azul, luego la roja
        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
            up[id] = cy
        if id in up:
            if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punto rojo
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                counter_up.append(id)  # lista

    # Dibujar líneas
    cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # línea roja
    cv2.line(frame, (0, 450), (850, 450), (255, 0, 0), 3)  # línea azul

    # Mostrar contadores en el frame
    downwards = len(set(counter_down))
    upwards = len(set(counter_up))

    print(f"Baixada: {downwards}")
    print(f"Pujada: {upwards}")

    # Mostrar el frame con las bounding boxes ajustadas
    fps += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    if elapsed_time > 0:
        current_fps = fps / elapsed_time
        cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 640x384 2 cars, 387.5ms
Speed: 22.2ms preprocess, 387.5ms inference, 20.9ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 2 cars, 175.0ms
Speed: 9.3ms preprocess, 175.0ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 2 cars, 168.6ms
Speed: 6.0ms preprocess, 168.6ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 2 cars, 168.7ms
Speed: 8.1ms preprocess, 168.7ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 2 cars, 171.5ms
Speed: 7.5ms preprocess, 171.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 2 cars, 167.0ms
Speed: 7.1ms preprocess, 167.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 2 cars, 176.6ms
Speed: 7.0ms preprocess, 176.6ms inference, 4.0ms postprocess per image at shape (1, 3, 

KeyboardInterrupt: 